In [ ]:
import pandas as pd
import json

In [ ]:
with open(r'C:\Users\comp\Documents\chicago_crime\walmarts_google_api_pg1') as f:
  p1 = json.load(f)
with open(r'C:\Users\comp\Documents\chicago_crime\walmarts_google_api_pg2') as f:
  p2 = json.load(f)
with open(r'C:\Users\comp\Documents\chicago_crime\walmarts_google_api_pg3') as f:
  p3 = json.load(f)

In [ ]:
df1 = pd.DataFrame(p1['results'])
df2 = pd.DataFrame(p2['results'])
df3 = pd.DataFrame(p3['results'])

In [ ]:
df_merge = pd.concat([df1, df2, df3])

In [ ]:
len(df_merge.index)

In [ ]:
df_merge

In [ ]:
import shapely
from shapely.ops import asShape
from shapely.geometry import Point

In [ ]:
df_merge.iloc[0,1]['location']

In [ ]:
Point(df_merge.iloc[0,1]['location']['lng'], df_merge.iloc[0,1]['location']['lat'])

In [ ]:
def parse_wm_point(g):
    p = Point(g['location']['lng'], g['location']['lat'])
    return WKTElement(p, srid=4326)

df_merge['geom'] = df_merge['geometry'].apply(lambda x : parse_wm_point(x))

In [ ]:
df_merge

In [ ]:
df_merge.dtypes

In [ ]:
from geoalchemy2 import Geometry, WKTElement
import  sqlalchemy 

pg = open('db_credentials.txt')
db = pg.read()
pg.close()

# load into db
engine = sqlalchemy.create_engine(db)
df_merge[['id','name','geom']].to_sql('walmart', engine, if_exists='replace', index=False, dtype={'geom': Geometry('POINT', srid= 4326)})

In [ ]:
with open(r'C:\Users\comp\Documents\chicago_crime\walmart_supercenter') as f:
  p1 = json.load(f)
with open(r'C:\Users\comp\Documents\chicago_crime\walmart_market') as f:
  p2 = json.load(f)

In [ ]:
df1 = pd.DataFrame(p1['results'])
df2 = pd.DataFrame(p2['results'])

df_merge = pd.concat([df1, df2])

df_merge['geom'] = df_merge['geometry'].apply(lambda x : parse_wm_point(x))

df_merge.drop_duplicates(subset = 'id', inplace=True)

In [ ]:
# load into db
df_merge[['id','name','geom']].to_sql('walmart', engine, if_exists='replace', index=False, dtype={'geom': Geometry('POINT', srid= 4326)})

In [ ]:
points = pd.read_csv('google_query_points_v2.csv')

In [ ]:
points

In [ ]:
import requests
import pandas as pd
import time

fin = open(r'C:\Users\comp\Documents\chicago_crime\google_maps_token.txt')
token = fin.read()
fin.close()

dist = 2800

for i, row in points.iterrows():
    print(i)
    x = row['X']
    y = row['Y']
    file_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=%.6f,%.6f&radius=%s&keyword=liquor+store&key=%s' % (y,x,dist,token)
 
    file_object = requests.get(file_url)

    with open('request_%d.txt' % (i), 'wb') as local_file:
        local_file.write(file_object.content)
        
    with open('request_%d.txt' % (i), 'r') as f:
      j1 = json.load(f)

    if i == 0:
        df = pd.DataFrame(j1['results'])
    else:
        df = pd.concat([df, pd.DataFrame(j1['results'])])
        
    n = 1
    while 'next_page_token' in j1.keys():
        time.sleep(5)
        file_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?key=%s&pagetoken=%s' % (token, j1['next_page_token'])
        print(file_url)
        file_object = requests.get(file_url)
        
        with open('request_%d_%d.txt' % (i,n), 'wb') as local_file:
            local_file.write(file_object.content)  
        with open('request_%d_%d.txt' % (i,n), 'r') as f:
          j1 = json.load(f)
    

        df = pd.concat([df, pd.DataFrame(j1['results'])])
        n += 1
        #break
#     if n > 1:
#         break
    time.sleep(5)
        
    

In [ ]:
df2 = df.drop_duplicates(subset = 'id')

In [ ]:
df2

In [ ]:
from shapely.geometry import Point

from geoalchemy2 import Geometry, WKTElement
import  sqlalchemy 

pg = open('db_credentials.txt')
db = pg.read()
pg.close()

df2['geom'] = df2['geometry'].apply(lambda x : parse_wm_point(x))
df2.drop(columns = ['geometry', 'opening_hours', 'photos', 'plus_code'], inplace = True)

# load into db
engine = sqlalchemy.create_engine(db)
df2.to_sql('liqour_stores', engine, if_exists='replace', index=False, dtype={'geom': Geometry('POINT', srid= 4326)})

In [ ]:
with open(r'request.txt') as f:
  j1 = json.load(f)

# j1 = json.load('request.txt')
p1 = pd.DataFrame(j1['results'])

In [ ]:
p1

In [ ]:
print(file_object.content)

In [ ]:
import json
json.load(file_object.content)